In [1]:
import os
import numpy as np
import pandas as pd
from uuid import uuid4


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("precision", 7)

In [1]:

PROCESSED_DATA_DIR = '../data/processed/'
# ground data file name
plot1_processed_forest_ground_data_with_agb_file = 'plot1_processed_forest_ground_data_with_agb.csv'
plot2_processed_forest_ground_data_with_agb_file = 'plot1_processed_forest_ground_data_with_agb.csv'

# aerial data file name
plot1_processed_aerial_data_file = 'plot1_processed_forest_aerial_data.csv'
plot2_processed_aerial_data_file = 'plot2_processed_forest_aerial_data.csv'

In [5]:
plot1_ground_data =pd.read_csv(PROCESSED_DATA_DIR  + plot1_processed_forest_ground_data_with_agb_file)
plot2_ground_data =pd.read_csv(PROCESSED_DATA_DIR  + plot2_processed_forest_ground_data_with_agb_file)

plot1_aerial_data=pd.read_csv(PROCESSED_DATA_DIR  + plot1_processed_aerial_data_file)
plot2_aerial_data=pd.read_csv(PROCESSED_DATA_DIR  + plot2_processed_aerial_data_file)

In [6]:
ground_data.head()

,tree_id,stem_id,species,stem_diameter,census_codes,tree_status,UTM_2018_X,UTM_2018_Y,latitude,longitude,genus,geo_index,agb,gdbin
0,1,1,libe,65,M,A,747387.5,4308579.5,38.8912770,-78.1476714,Lindera,747000_4308000,3519.5723873,4
1,2,2,libe,21,M,A,747401.4,4308565.7,38.8911491,-78.1475160,Lindera,747000_4308000,192.9561904,2
2,3,3,libe,32,DS,G,747389.9,4308703.7,38.8923941,-78.1475988,Lindera,747000_4308000,1061.8221206,3
3,4,4,nysy,142,M,A,747389.8,4308935.2,38.8944773,-78.1475173,Nyssa,747000_4308000,41008.0807715,5
4,5,5,havi,96,M;P,A,747384.8,4308942.9,38.8945488,-78.1475719,Hamamelis,747000_4308000,19509.6480279,5


In [7]:
aerial_data.head()

,Unnamed: 0,treeID,stemID,tag,StemTag,sp,quadrat,gx,gy,MeasureID,CensusID,dbh,hom,ExactDate,DFstatus,codes,date,status,Genus,Species,agb
0,1,1,1,10079,1,libe,104,3.7,73.00000,233097,3,65,1.3,8/27/2018,alive,M,21423,A,Lindera,benzoin,3179.4770053
1,2,2,2,10168,1,libe,103,17.3,58.90000,232970,3,21,1.3,8/27/2018,alive,M,21423,A,Lindera,benzoin,192.9561910
2,3,3,3,10567,1,libe,110,9.0,197.10001,234079,3,32,1.3,8/30/2018,stem dead,DS,21426,G,Lindera,benzoin,548.4256425
3,4,4,4,12165,1,nysy,122,14.2,428.50000,234467,3,142,1.3,8/29/2018,alive,M,21425,A,Nyssa,sylvatica,22164.5610597
4,5,5,5,12190,1,havi,122,9.4,436.39999,234505,3,96,1.3,8/29/2018,alive,M;P,21425,A,Hamamelis,virginiana,8414.7345729


In [44]:
import itertools
import time
import numpy as np
import pandas as pd
import os
from threading import Thread, Event
from multiprocessing import Process, Event  
from multiprocessing import synchronize  



ground_data =pd.read_csv(PROCESSED_DATA_DIR  + processed_forest_ground_data_with_agb_file)
aerial_data=pd.read_csv(PROCESSED_DATA_DIR  + processed_aerial_data_file)
ground_data.stem_diameter = ground_data.stem_diameter.astype('float')

def isground_tree_inboundingbox(bb_x1,bb_x2,bb_y1, bb_y2, gt_X, gt_Y):

    if (gt_X >= bb_x1 and gt_X <= bb_x2) and (gt_Y >= bb_y1 and gt_Y <= bb_y2):
        return True
    else:
        return False

def merge_aerial_ground_treedata(ar_row, grnd_row):
    
    candidate_matches = pd.DataFrame(columns =[
        'ground_data_tree_id', 'aerial_data_tree_id', 'stem_id','species', 'stem_diameter', 'agb','score', 'aerial_data_height',
        'area' , 'UTM_2018_X', 'UTM_2018_Y', 'left','bottom', 'right', 'top', 'tree_status', 'census_codes', 'adbin',  'gdbin'])
    

    candidate_matches = candidate_matches.append({'ground_data_tree_id' : grnd_row.tree_id,
                                        'aerial_data_tree_id' : ar_row.id,
                                        'stem_id' : grnd_row.stem_id,
                                        'species' : grnd_row.species,
                                        'stem_diameter' : grnd_row.stem_diameter,
                                        'agb': grnd_row.agb,
                                        'score': ar_row.score, #matched_tree['score'].iloc[0],
                                        'aerial_data_height': np.round(ar_row.height,7), #matched_tree['height'].iloc[0],
                                        'area': ar_row.area, #matched_tree['area'].iloc[0],
                                        'UTM_2018_X': grnd_row.UTM_2018_X,
                                        'UTM_2018_Y': grnd_row.UTM_2018_Y,
                                        'left': ar_row.left, #matched_tree['left'].iloc[0],
                                        'bottom': ar_row.bottom, # matched_tree['bottom'].iloc[0],
                                        'right': ar_row.right, #matched_tree['right'].iloc[0],
                                        'top': ar_row.top, #matched_tree['top'].iloc[0],
                                        'tree_status': grnd_row.tree_status,
                                        'census_codes': grnd_row.census_codes,
                                        'adbin': ar_row.adbin,
                                        'gdbin': grnd_row.gdbin,
                                       }, ignore_index=True)
        

    return candidate_matches

def match_trees(geo_index, aerial_data, ground_data, file_name, lower_quantile):
    """
    Matching Algorithm
    """
    

    geo_index_aerial_data = aerial_data[aerial_data['geo_index'] == geo_index]
    geo_index_ground_data = ground_data[ground_data['geo_index'] == geo_index]
    
    #dbh_filter= geo_index_ground_data.stem_diameter.quantile(lower_quantile)
    
    #print(f"\n Num. trees in ground_data with geo_index {geo_index} before filtering {lower_quantile} - {dbh_filter} lower_quantile:  {geo_index_ground_data.shape[0]}")
    #geo_index_ground_data = geo_index_ground_data.loc[(geo_index_ground_data.stem_diameter > dbh_filter)] 
    #print(f"\n Num. trees in ground_data with geo_index {geo_index} after filtering {lower_quantile} - {dbh_filter}  lower_quantile:  {geo_index_ground_data.shape[0]}")
    
    #create and empty dataframe object
    matched_df = pd.DataFrame(columns =[
        'ground_data_tree_id', 'aerial_data_tree_id', 'stem_id','species', 'stem_diameter', 'agb', 'score', 'aerial_data_height', 
        'area' , 'UTM_2018_X', 'UTM_2018_Y', 'left','bottom', 'right', 'top',  'tree_status','census_codes', 'adbin','gdbin'])
    
    
    for ar_row in geo_index_aerial_data.itertuples():

        candidate_matches = pd.DataFrame(columns =[
        'ground_data_tree_id', 'aerial_data_tree_id', 'stem_id','species', 'stem_diameter', 'agb', 'score', 'aerial_data_height',  
        'area' , 'UTM_2018_X', 'UTM_2018_Y', 'left','bottom', 'right', 'top',  'tree_status','census_codes', 'adbin','gdbin'])
    
        #print(f'Num rows in ground_data {geo_index_ground_data.shape[0]}')
        for grnd_row in geo_index_ground_data.itertuples():

            is_tree_in_bb = isground_tree_inboundingbox(
                ar_row.left, 
                ar_row.right, 
                ar_row.bottom,
                ar_row.top,
                grnd_row.UTM_2018_X,
                grnd_row.UTM_2018_Y)
            
            is_bin_matching = True if ar_row.adbin == grnd_row.gdbin else False
            
            if is_tree_in_bb == True and is_bin_matching == True:
                candidate_row = merge_aerial_ground_treedata(ar_row=ar_row, grnd_row=grnd_row)
                candidate_matches = candidate_matches.append(candidate_row)

        if len(candidate_matches) > 1 :
            matched_df = matched_df.append(candidate_matches[candidate_matches.agb == candidate_matches.agb.max()])
            tree_id = candidate_matches.nlargest(1, 'aerial_data_height').ground_data_tree_id[0]

            # removing the matched tree from the ground dataset
            geo_index_ground_data = geo_index_ground_data[geo_index_ground_data.tree_id != tree_id]
        elif len(candidate_matches) == 1:  
            matched_df = matched_df.append(candidate_matches)
            tree_id = candidate_matches.iloc[0,0]
            # removing the matched tree from the ground dataset
            geo_index_ground_data = geo_index_ground_data[geo_index_ground_data.tree_id != tree_id]

    matched_df = matched_df.reset_index(drop=True)
    print(matched_df.shape)
    matched_df.to_csv(file_name, header=True, index=False)
    return matched_df



In [45]:
df1 = match_trees('747000_4308000',aerial_data, ground_data, '../data/matched_trees1.csv', [0.1])


(2671, 19)


In [41]:
df2 = match_trees('747000_4309000',aerial_data, ground_data, '../data/matched_trees2.csv', [0.1])

(26, 19)


In [46]:
matched_trees = df1.append(df2)
matched_trees.to_csv('../data/processed/filter_exact_match_trees.csv', header=True, index=False)

AttributeError: 'NoneType' object has no attribute 'append'

In [ ]:
def main() -> '__main__':

    stem_dbh_filters = [0.1]#,0.2,0.3]
    
    for lower_quantile in stem_dbh_filters:
        print(f" begin matching process for stem_diameters > {lower_quantile}% quantile")
        process1 = Process(target=match_trees, args=('747000_4308000',aerial_data, ground_data, 'data/matched/matched_trees1.csv', lower_quantile))  
        process2 = Process(target=match_trees, args=('747000_4309000',aerial_data, ground_data, 'data/matched/matched_trees2.csv', lower_quantile))  
               
        process1.start() 
        process2.start()  
    
        process1.join()  
        process2.join()  
    
        matched_trees1 = pd.read_csv('data/matched/matched_trees1.csv')
        matched_trees2 = pd.read_csv('data/matched/matched_trees2.csv')
        matched_trees = matched_trees1.append(matched_trees2)
        matched_trees.to_csv('data/matched/filter-'+ str(lower_quantile) + '_matched_trees.csv', header=True, index=False)
        print(f" End matching process for stem_diameters > {lower_quantile}% quantile")

if __name__ == '__main__':
    main()

In [ ]:
dbh_filter= ground_data.stem_diameter.quantile(0.3)

In [ ]:
ground_data.loc[(ground_data.stem_diameter > dbh_filter)].shape[0]

In [ ]:
csv1 = pd.read_csv('data/matched/filter-0.1_matched_trees.csv')

In [ ]:
plt.scatter(csv1.area, csv1.stem_diameter, alpha=0.2, cmap='viridis')
plt.xlabel('area')
plt.ylabel('dbh');
plt.show()